In [6]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, mannwhitneyu, wilcoxon, ttest_rel, ttest_ind
import seaborn as sns
from matplotlib.patches import FancyArrowPatch
from matplotlib.colors import ColorConverter


PROJECT_PATH = os.getcwd()
proto_path = os.path.abspath(os.path.join(PROJECT_PATH, os.pardir))
neuro_path = os.path.abspath(os.path.join(proto_path, os.pardir))
sys.path.append(neuro_path)
print(neuro_path)

from library.study_space import Session, Study
from x_io.rw.axona.batch_read import make_study
from _prototypes.cell_remapping.src.masks import make_object_ratemap

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit


In [8]:
STUDY_SETTINGS = {

        'ppm': 485,  # EDIT HERE

        'smoothing_factor': 3, # EDIT HERE

        'useMatchedCut': True,  # EDIT HERE
    }


# Switch devices to True/False based on what is used in the acquisition (to be extended for more devices in future)
device_settings = {'axona_led_tracker': True, 'implant': True} 
# Make sure implant metadata is correct, change if not, AT THE MINIMUM leave implant_type: tetrode
implant_settings = {'implant_type': 'tetrode', 'implant_geometry': 'square', 'wire_length': 25, 'wire_length_units': 'um', 'implant_units': 'uV'}
# WE ASSUME DEVICE AND IMPLANT SETTINGS ARE CONSISTENCE ACROSS SESSIONS
# Set channel count + add device/implant settings
SESSION_SETTINGS = {
    'channel_count': 4, # EDIT HERE, default is 4, you can change to other number but code will check how many tetrode files are present and set that to channel copunt regardless
    'devices': device_settings, # EDIT HERE
    'implant': implant_settings, # EDIT HERE
}
STUDY_SETTINGS['session'] = SESSION_SETTINGS
settings_dict = STUDY_SETTINGS

settings_dict['speed_lowerbound'] = 0 
settings_dict['speed_upperbound'] = 99
settings_dict['ratemap_dims'] = (32,32)
settings_dict['disk_arena'] = True

NameError: name 'settings_dict' is not defined

In [ ]:
def flat_disk_mask(rate_map):
    masked_rate_map = disk_mask(rate_map)
    masked_rate_map.data[masked_rate_map.mask] = np.nan
    return  masked_rate_map.data

In [ ]:
folder_path = '...'

output_folder_path = '...'

dict_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\_prototypes\cell_remapping\identifier_dict.pkl"

# list all folders in folder_path at any level  
# folder_list = [x[0] for x in os.walk(folder_path)]

def _find_subdir(folder_list,aid,date):
    """
    Finds the subdirectory of a given animal and date
    """
    for folder in folder_list:
        if date in str(folder):
            if aid in str(folder.split('\\')[-2]):
                return folder

ctype_dict = pd.read_pickle(dict_path)

for ctype in ctype_dict:
    
    for group in ctype_dict[ctype]:

        animal_cell_list =  ctype_dict[ctype][group]
        
        for animal in animal_cell_list:
            aid = animal[1]
            date = animal[3]
            tetrode = animal[4]
            cell = animal[5]
            

            settings_dict['single_tet'] = int(tetrode)

            subdir = _find_subdir(folder_list,aid,date)

            sub_study = make_study(subdir,settings_dict=settings_dict)
            sub_study.make_animals()       

            animal = sub_study.animals[0]

            ses_ratemaps = []
            ses_ratemap_objs = []

            for session in animal.sessions:
                pos_obj = session.get_position_data()['position']

                tet_path = session.session_metadata.file_paths['tet']
                fname = tet_path.split('/')[-1].split('.')[0]

                if settings['disk_arena']: 
                    cylinder = True
                else:
                    cylinder, _ = check_disk_arena(fname)
                    if not cylinder:
                        print('Not cylinder for {}'.format(fname))
                        
                if settings['naming_type'] == 'LEC':
                    group, name = extract_name_lec(fname)
                    formats = LEC_naming_format[group][name][settings['type']]

                for format in list(formats.keys()):
                    checked = _check_single_format(fname, format, formats[format])
                    if checked is not None:
                        break
                    else:
                        continue
                
                angle, depth, name, date = checked

                spatial_spike_train = session.make_class(SpatialSpikeTrain2D, 
                    {'cell': cell, 'position': pos_obj, 'speed_bounds': (settings_dict['speed_lowerbound'], settings_dict['speed_upperbound'])})   

                for cell in session.get_cell_data()['cell_ensemble'].cells:  
                    if cell_label in ensemble.get_cell_label_dict():
        
                        rate_obj = spatial_spike_train.get_map('rate')
                        rate_map, rate_map_raw = rate_obj.get_rate_map(new_size=settings_dict['ratemap_dims'][0])

                        if cylinder:
                            curr = flat_disk_mask(rate_map)
                            
                        ses_ratemaps.append(rate_map)
                        ses_ratemap_objs.append(rate_obj)

            fig = plt.figure(figsize=(10,10))

            for i, ratemap in enumerate(ses_ratemaps):
                ax = fig.add_subplot(1,len(ses_ratemaps),i+1)
                img = ax.imshow(ratemap, cmap='jet')
                ax.set_title(angle)

                _, obj_loc = make_object_ratemap(angle, ses_ratemap_objs[i])

                ax.plot(obj_loc[0], obj_loc[1], 'kx', markersize=10)

                fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

            fig.tight_layout()
            plt.show()
            stop()
            # save to output folder
            fig.savefig(output_folder_path, dpi=360)
            plt.close()
            



                

NameError: name 'settings_dict' is not defined